# **Gaussian Elimination Steps**


In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np

# ==========================================
# 1. SOLVER ENGINE (Updated for Clean Text)
# ==========================================
class SolverState:
    def __init__(self, matrix, message, math_html, cost_ops, cost_mult, cost_add, pivot_pos=None, target_pos=None):
        self.matrix = matrix.copy()
        self.message = message
        self.math_html = math_html  # Now storing ready-to-render HTML
        self.cost = (cost_ops, cost_mult, cost_add)
        self.pivot_pos = pivot_pos
        self.target_pos = target_pos

def solve_gaussian(matrix_in):
    mat = np.array(matrix_in, dtype=float)
    steps = []
    rows, cols = mat.shape
    c_ops = c_mult = c_add = 0

    # Initial State
    steps.append(SolverState(mat, "Start: Initial Augmented Matrix", "Ax = b", 0,0,0))

    # --- FORWARD ELIMINATION ---
    for i in range(rows):
        # 1. Pivot Check
        pivot = mat[i, i]
        steps.append(SolverState(mat, f"Checking Pivot at ({i+1},{i+1})", f"Pivot = {pivot:.2g}", c_ops, c_mult, c_add, pivot_pos=(i,i)))

        if abs(pivot) < 1e-10:
            # Swap logic
            found = False
            for k in range(i + 1, rows):
                if abs(mat[k, i]) > 1e-10:
                    mat[[i, k]] = mat[[k, i]]
                    # HTML Math: R1 <-> R2
                    math_txt = f"R<sub>{i+1}</sub> ↔ R<sub>{k+1}</sub>"
                    steps.append(SolverState(mat, f"Swapped Row {i+1} with Row {k+1}", math_txt, c_ops, c_mult, c_add, pivot_pos=(i,i)))
                    pivot = mat[i, i]
                    found = True
                    break
            if not found:
                steps.append(SolverState(mat, "Matrix is Singular", "<span style='color:red'>FAILURE: Pivot is Zero</span>", c_ops, c_mult, c_add))
                return steps

        # 2. Elimination
        for j in range(i + 1, rows):
            target = mat[j, i]
            if abs(target) > 1e-10:
                multiplier = target / pivot
                mat[j] = mat[j] - multiplier * mat[i]

                c_ops += 1; c_mult += 4; c_add += 4

                msg = f"Eliminating ({j+1},{i+1})"
                # HTML Math: R2 <- R2 - (m)R1
                math_txt = f"R<sub>{j+1}</sub> ← R<sub>{j+1}</sub> - ({multiplier:.2g}) · R<sub>{i+1}</sub>"

                steps.append(SolverState(mat, msg, math_txt, c_ops, c_mult, c_add, pivot_pos=(i,i), target_pos=(j,i)))

    # --- BACK SUBSTITUTION ---
    x = np.zeros(3)
    if abs(mat[2, 2]) > 1e-10:
        # Z
        x[2] = mat[2, 3] / mat[2, 2]
        math_txt = f"z = {mat[2,3]:.2g} / {mat[2,2]:.2g} = <b>{x[2]:.3g}</b>"
        steps.append(SolverState(mat, f"Solving Z (Row 3)", math_txt, c_ops, c_mult, c_add, pivot_pos=(2,2)))

        # Y
        x[1] = (mat[1, 3] - mat[1, 2]*x[2]) / mat[1, 1]
        math_txt = f"y = ({mat[1,3]:.2g} - {mat[1,2]:.2g}z) / {mat[1,1]:.2g} = <b>{x[1]:.3g}</b>"
        steps.append(SolverState(mat, f"Solving Y (Row 2)", math_txt, c_ops, c_mult, c_add, pivot_pos=(1,1)))

        # X
        x[0] = (mat[0, 3] - mat[0, 1]*x[1] - mat[0, 2]*x[2]) / mat[0, 0]
        math_txt = f"x = ({mat[0,3]:.2g} - ... ) / {mat[0,0]:.2g} = <b>{x[0]:.3g}</b>"
        steps.append(SolverState(mat, f"Solving X (Row 1)", math_txt, c_ops, c_mult, c_add, pivot_pos=(0,0)))

        # Final
        final_res = f"x = {x[0]:.3g}, &nbsp; y = {x[1]:.3g}, &nbsp; z = {x[2]:.3g}"
        steps.append(SolverState(mat, "✅ Solution Found", final_res, c_ops, c_mult, c_add))
    else:
        steps.append(SolverState(mat, "Singular Matrix", "Infinite or No Solution", c_ops, c_mult, c_add))

    return steps

# ==========================================
# 2. UI STYLING & HELPERS
# ==========================================
CSS = """
<style>
    .dash-container { font-family: 'Segoe UI', Helvetica, sans-serif; max-width: 750px; background: #ffffff; padding: 25px; border: 1px solid #e0e0e0; border-radius: 12px; box-shadow: 0 4px 12px rgba(0,0,0,0.08); }
    .header { background: linear-gradient(135deg, #4b6cb7 0%, #182848 100%); color: white; padding: 20px; border-radius: 8px 8px 0 0; text-align: center; margin: -25px -25px 20px -25px; }

    .matrix-box { display: grid; grid-template-columns: repeat(4, 1fr); gap: 8px; background: #f8f9fa; padding: 15px; border-radius: 10px; width: 400px; margin: auto; border: 1px solid #ddd; }
    .cell { background: white; padding: 12px; text-align: center; border-radius: 6px; font-weight: 600; box-shadow: 0 2px 4px rgba(0,0,0,0.05); font-size: 16px; color: #333; }

    /* Dynamic Highlights */
    .cell-pivot { background-color: #d1e7dd !important; border: 2px solid #198754; color: #0f5132; transform: scale(1.05); transition: 0.2s; }
    .cell-target { background-color: #f8d7da !important; border: 2px solid #dc3545; color: #842029; }

    .info-card { background: #e3f2fd; padding: 20px; border-left: 6px solid #2196f3; margin: 20px 0; border-radius: 6px; }
    .action-label { font-size: 12px; color: #555; text-transform: uppercase; letter-spacing: 1px; font-weight: bold; margin-bottom: 5px; }
    .action-text { font-size: 18px; font-weight: bold; color: #0d47a1; margin-bottom: 8px; }
    .math-disp { font-family: 'Consolas', 'Monaco', monospace; font-size: 18px; color: #333; background: rgba(255,255,255,0.6); padding: 8px; border-radius: 4px; display: inline-block; }

    .cost-box { display: grid; grid-template-columns: 1fr 1fr 1fr; gap: 10px; margin-top: 20px; text-align: center; }
    .cost-item { background: #fff3cd; padding: 10px; border-radius: 8px; border: 1px solid #ffeeba; font-size: 14px; font-weight: bold; color: #856404; }
</style>
"""

# ==========================================
# 3. DASHBOARD LOGIC
# ==========================================

# --- INPUT SECTION ---
input_widgets = []
# Default system: 2x + y - z = 8 ...
default_vals = [2, 1, -1, 8, -3, -1, 2, -11, -2, 1, 2, -3]
grid_layout = widgets.GridspecLayout(3, 4, width='450px')

idx = 0
for r in range(3):
    for c in range(4):
        w = widgets.FloatText(value=default_vals[idx], layout=widgets.Layout(width='95%'))
        grid_layout[r, c] = w
        input_widgets.append(w)
        idx += 1

btn_run = widgets.Button(
    description="▶ START SIMULATION",
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width='100%', margin='15px 0')
)

output_display = widgets.Output()
history = []

def render_step(step_idx):
    if not history: return

    current = history[step_idx]
    mat = current.matrix
    piv = current.pivot_pos
    tar = current.target_pos

    # 1. Build Matrix Grid HTML
    grid_html = '<div class="matrix-box">'
    for r in range(3):
        for c in range(4):
            val = mat[r,c]

            # Formatting: Handle -0.0 and precision
            txt = f"{val:.4g}" if abs(val)>1e-10 else "0"

            css_class = "cell"
            if piv and r == piv[0] and c == piv[1]: css_class += " cell-pivot"
            if tar and r == tar[0] and c == tar[1]: css_class += " cell-target"

            # Vertical bar for augmented matrix
            style = "border-right: 3px solid #bbb;" if c == 2 else ""

            grid_html += f'<div class="{css_class}" style="{style}">{txt}</div>'
    grid_html += '</div>'

    # 2. Build Full Dashboard HTML
    html = f"""
    {CSS}
    <div class="dash-container">
        <div class="header">
            <h2 style="margin:0">🎓 Gaussian Elimination Lab</h2>
        </div>

        <div class="info-card">
            <div class="action-label">Current Operation</div>
            <div class="action-text">{current.message}</div>
            <div class="math-disp">{current.math_html}</div>
        </div>

        {grid_html}

        <div class="cost-box">
            <div class="cost-item">Rows Modified<br>{current.cost[0]}</div>
            <div class="cost-item">Multiplications<br>{current.cost[1]}</div>
            <div class="cost-item">Additions<br>{current.cost[2]}</div>
        </div>
    </div>
    """

    with output_display:
        clear_output(wait=True)
        display(widgets.HTML(html))

# --- CONTROLLER ---
slider = widgets.IntSlider(min=0, max=0, step=1, description='Step:', layout=widgets.Layout(width='95%'), disabled=True)

def on_run_click(b):
    global history
    raw_vals = [w.value for w in input_widgets]
    matrix_in = np.array(raw_vals).reshape(3, 4)

    history = solve_gaussian(matrix_in)

    slider.max = len(history) - 1
    slider.value = 0
    slider.disabled = False
    render_step(0)

def on_slider_change(change):
    render_step(change['new'])

btn_run.on_click(on_run_click)
slider.observe(on_slider_change, names='value')

# --- DISPLAY ---
input_header = widgets.HTML("<h4>1. Edit Matrix Coefficients (A | b)</h4>")
input_area = widgets.VBox([input_header, grid_layout, btn_run], layout=widgets.Layout(border='1px solid #ddd', padding='15px', margin='0 0 20px 0', width='500px'))
app = widgets.VBox([input_area, slider, output_display])

display(app)